In [1]:
import re
import pandas as pd
import numpy as np
import csv
import json
import os

In [2]:
os.getcwd()

'C:\\Users\\clu\\Documents\\cal poly\\research\\Kelly Bodwin\\notebooks\\final drafts'

In [4]:
df = pd.read_csv(open("..\\..\\data\\Weekly_data.csv", encoding = "utf-8"))

In [5]:
df.head()

,Artists,Name,Weekly rank,Peak position,Weeks on chart,Week,Date,Genre,Writing Credits,Lyrics
0,Drake,In My Feelings,1,1.0,5.0,2018-08-11,"June 29, 2018",NaN,"Phil triggaman price, Orville bugs can can hal...",In My Feelings Lyrics\n\n\n\n[Intro: Drake]\nT...
1,6ix9ine Featuring Nicki Minaj & Murda Beatz,FEFE,3,3.0,2.0,2018-08-11,"July 22, 2018",NaN,"Baka, Cubeatz, Shotti3, Nicki minaj, Murda beatz","FEFE Lyrics\n\n\n\n[Intro: 6ix9ine, Nicki Mina..."
2,Maroon 5 Featuring Cardi B,Girls Like You,4,3.0,10.0,2018-08-11,"May 30, 2018",NaN,"Gian stone, Cirkut, Cardi b, Starrah, Adam lev...",Girls Like You (Remix) Lyrics\n\n\n\n[Verse 1:...
3,"DJ Khaled Featuring Justin Bieber, Chance The ...",No Brainer,5,NaN,NaN,2018-08-11,"July 27, 2018",NaN,"Dj khaled, Justin bieber, Quavo, Chance the ra...",No Brainer Lyrics\n\n\n\n[Intro: DJ Khaled]\nW...
4,Post Malone,Better Now,6,5.0,14.0,2018-08-11,"April 27, 2018",NaN,"Kaan gunesberk, William walsh, Carl rosen, Bil...",Better Now Lyrics\n\n\n\n[Chorus]\nYou prolly ...


In [2]:
df = pd.read_csv(open("..//..//data//Weekly_data_clean.csv", encoding = "utf-8"))

In [3]:
df.head()

,Artists,Name,Weekly rank,Peak position,Weeks on chart,Week,Date,Genre,Writing Credits,Lyrics,Songwriter
0,Drake,In My Feelings,1,1.0,5.0,2018-08-11,"June 29, 2018",NaN,"Phil triggaman price, Orville bugs can can hal...","In My Feelings \nTrap, TrapMoneyBenny\nThis sh...",False
1,"6ix9ine, Nicki Minaj, Murda Beatz",FEFE,3,3.0,2.0,2018-08-11,"July 22, 2018",NaN,"Baka, Cubeatz, Shotti3, Nicki minaj, Murda beatz",FEFE \nIt's fucking TR3YWAY!\nKing of New York...,False
2,"Maroon 5, Cardi B",Girls Like You,4,3.0,10.0,2018-08-11,"May 30, 2018",NaN,"Gian stone, Cirkut, Cardi b, Starrah, Adam lev...","Girls Like You Remix \nSpent 24 hours, I need ...",True
3,"DJ Khaled, Justin Bieber, Chance The Rapper, Q...",No Brainer,5,NaN,NaN,2018-08-11,"July 27, 2018",NaN,"Dj khaled, Justin bieber, Quavo, Chance the ra...",No Brainer \nWe the Best Music!\nAnother one!\...,False
4,Post Malone,Better Now,6,5.0,14.0,2018-08-11,"April 27, 2018",NaN,"Kaan gunesberk, William walsh, Carl rosen, Bil...",Better Now \nYou prolly think that you are bet...,False


In [5]:
def isnull(x):
    """
    """
    return x != x

def clean_lyric(lyric, uni_replace):
    
    repl = {"\[.+?\]": "", 
            "\s{2,}": "\n", 
            "lyrics": "", 
            "\(": "", 
            "\)": "",
            "\<u\+0092\>": "'",
            "\<u\+0097\>": "-",
            "\<u\+00..\>": "",
            " '{0,}": " ",
            "in'{1} ": "ing ",
            "\.{0,} ": " ",
            " \.{0,}": " ",
            ' "{0,}': " ",
            '"{0,} ': " ",
            }
    
    for i in repl:
        lyric = re.sub(i, repl[i], lyric, flags = re.I | re.S)
        
    for i in uni_replace:
        lyric = re.sub(i, "", lyric, flags = re.I | re.S)
        
    
    
    return lyric

In [7]:
filler = [" and ", " featuring ", " & ", " x ", " / "]
repl = {"(\"|”).+?(\"|”)": "", "\s+": " ", "\s,": ","}
textcols = ["Artists", "Name", "Genre", "Writing Credits", "Lyrics"]
unicode_replace  = json.load(open("..//..//data//unicoderemove.json", "r"))

for i in textcols:
    for j in range(len(df)):
        if not isnull(df.loc[j, i]):
            if i == "Artists":
                for k in filler:
                    df.loc[j, i] = re.sub(k, ", ", df.loc[j, i], flags = re.I | re.S)
            elif i == "Lyrics":
                df.loc[j, i] = clean_lyric(df.loc[j, i], unicode_replace)
            else:
                for k in repl:
                    df.loc[j, i] = re.sub(k, repl[k], df.loc[j, i], flags = re.I | re.S)
                df.loc[j, i] = df.loc[j, i].strip()

In [25]:
# df[df.Name == "No Lie"].reset_index(drop = True).loc[0].Lyrics

## Add songwriters

In [8]:
writers = json.load(open("..//..//data//composers.json", "r"))

In [9]:
composers = []
for i in range(len(df)):
    for j in writers:
        if not isnull(df.loc[i, "Writing Credits"]):
            if re.search(j, df.loc[i, "Writing Credits"], flags = re.I | re.S) is not None:
                if re.search(j, df.loc[i, "Artists"], flags = re.I | re.S) is None:
                    composers.append(True)
                    break
    else:
        composers.append(False)

In [10]:
len(composers)

49500

In [11]:
df["Songwriter"] = np.array(composers)

# we drop NA's here for convenience now....will fix later
df = df[df.Lyrics == df.Lyrics]
df.to_csv("..//..//data//Weekly_data_clean.csv", index = False)